In [2]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import ollama
from concurrent.futures import ThreadPoolExecutor, as_completed, TimeoutError
from itertools import islice
import atexit
import requests

BASE = Path(".")
RESUME_PARQUET = BASE / "processed" / "resume_matched.parquet"
JOB_PARQUET    = BASE / "processed" / "dice_job_descriptions_matched.parquet"
MATCH_DIR      = BASE / "matches"
CACHE_FILE     = MATCH_DIR / "skill_cache.json"

MATCH_DIR.mkdir(exist_ok=True)

EMBED_MODEL = "nomic-embed-text"

In [10]:
resumes = pd.read_parquet(RESUME_PARQUET)
jobs    = pd.read_parquet(JOB_PARQUET)

resumes = resumes[resumes['Category'] == 'INFORMATION-TECHNOLOGY']

print(f"Loaded {len(resumes):,} resumes and {len(jobs):,} jobs")
print("Resume columns:", resumes.columns.tolist())
print("Job columns:   ", jobs.columns.tolist())

Loaded 120 resumes and 22,000 jobs
Resume columns: ['ID', 'Resume_str', 'Resume_html', 'Category', 'extracted_skills', 'length', 'matched_skills']
Job columns:    ['advertiserurl', 'company', 'employmenttype_jobstatus', 'jobdescription', 'jobid', 'joblocation_address', 'jobtitle', 'postdate', 'shift', 'site_name', 'skills', 'uniq_id', 'extracted_skills', 'matched_skills']


In [11]:
def get_esco_skills(row):
    """
    Extract ESCO labels from 'matched_skills' column.
    Works with: list, np.ndarray, tuple, str, None, NaN.
    """
    if hasattr(row, '_fields'):
        raw = getattr(row, 'matched_skills', None)
    else:
        raw = row.get('matched_skills', None)

    if raw is None:
        return []

    if isinstance(raw, float) and pd.isna(raw):
        return []

    if isinstance(raw, (list, np.ndarray, tuple)):
        return [str(s).strip() for s in raw if str(s).strip()]

    if isinstance(raw, str):
        import ast
        try:
            parsed = ast.literal_eval(raw)
            if isinstance(parsed, (list, tuple, np.ndarray)):
                return [str(s).strip() for s in parsed if str(s).strip()]
        except:
            pass
        return [s.strip() for s in raw.split(',') if s.strip()]

    return []

# Test
print("Test get_esco_skills:")
for i in range(min(3, len(resumes))):
    skills = get_esco_skills(resumes.iloc[i])
    print(f"  Resume {i}: {len(skills)} skills → {skills[:2]}")

Test get_esco_skills:
  Resume 0: 6 skills → ['Microsoft Visio', 'establish collaborative relations']
  Resume 1: 21 skills → ['update budget', 'Cisco']
  Resume 2: 11 skills → ['Cisco', 'assist customers']


In [12]:
skill_cache = {}
if CACHE_FILE.exists():
    print(f"Loading cache with {len(json.load(open(CACHE_FILE)))} skills...")
    skill_cache = {k: np.array(v) for k, v in json.load(open(CACHE_FILE)).items()}

def embed_skill(skill, timeout=30):
    skill = skill.strip()
    if skill in skill_cache:
        return skill_cache[skill]
    
    try:
        response = requests.post(
            "http://localhost:11434/api/embeddings",
            json={"model": EMBED_MODEL, "prompt": skill},
            timeout=timeout
        )
        response.raise_for_status()
        vec = np.array(response.json()["embedding"])
        skill_cache[skill] = vec
        return vec
    except requests.Timeout:
        print(f"[TIMEOUT] Embedding '{skill}' took >{timeout}s")
        return None
    except Exception as e:
        print(f"[ERROR] Embedding '{skill}' failed: {e}")
        return None

def embed_list(skills):
    vectors = []
    for s in skills:
        vec = embed_skill(s)
        if vec is not None:
            vectors.append(vec)
    return np.vstack(vectors) if vectors else np.array([]).reshape(0, 768)

def save_cache():
    print(f"Saving {len(skill_cache):,} skills to {CACHE_FILE}")
    json.dump({k: v.tolist() for k, v in skill_cache.items()}, open(CACHE_FILE, "w"))
atexit.register(save_cache)

Loading cache with 2604 skills...


<function __main__.save_cache()>

In [13]:
def match_resume_job(resume_row, job_row, thresh=0.55):
    r_skills = get_esco_skills(resume_row)
    j_skills = get_esco_skills(job_row)
    if not r_skills or not j_skills:
        return None

    r_emb = embed_list(r_skills)
    j_emb = embed_list(j_skills)
    if r_emb.size == 0 or j_emb.size == 0:
        return None

    sim = cosine_similarity(r_emb, j_emb)

    matches = []
    for i, rs in enumerate(r_skills):
        for j, js in enumerate(j_skills):
            if sim[i, j] >= thresh:
                matches.append({
                    "resume_skill": rs,
                    "job_skill": js,
                    "sim": float(sim[i, j])
                })

    matches_df = pd.DataFrame(matches).sort_values("sim", ascending=False) if matches else pd.DataFrame(columns=["resume_skill", "job_skill", "sim"])
    best_per_r = sim.max(axis=1)
    matched_job = set(matches_df["job_skill"]) if not matches_df.empty else set()
    gaps = [s for s in j_skills if s not in matched_job]

    summary = {
        "resume_id": resume_row.ID if hasattr(resume_row, 'ID') else resume_row.get('ID'),
        "job_id": job_row.jobid if hasattr(job_row, 'jobid') else job_row.get('jobid'),
        "n_r": len(r_skills),
        "n_j": len(j_skills),
        "avg_match": float(best_per_r.mean()) if len(best_per_r) > 0 else 0.0,
        "pct_covered": float((best_per_r >= thresh).mean()) if len(best_per_r) > 0 else 0.0,
        "n_matches": len(matches),
        "gaps": gaps
    }

    return {
        "summary": summary,
        "matches": matches_df,
        "sim_matrix": sim.tolist(),
        "r_skills": r_skills,
        "j_skills": j_skills
    }

In [14]:
r_idx = next((i for i in range(len(resumes)) if len(get_esco_skills(resumes.iloc[i])) > 0), None)
j_idx = next((i for i in range(len(jobs)) if len(get_esco_skills(jobs.iloc[i])) > 0), None)

if r_idx is not None and j_idx is not None:
    print(f"Testing Resume ID={resumes.iloc[r_idx]['ID']} ↔ Job ID={jobs.iloc[j_idx]['jobid']}")
    demo = match_resume_job(resumes.iloc[r_idx], jobs.iloc[j_idx], thresh=0.55)
    if demo:
        print("MATCH SUMMARY:")
        print(pd.Series(demo["summary"]))
        if not demo["matches"].empty:
            print("\nTOP 3 MATCHES:")
            display(demo["matches"].head(3))
    else:
        print("No result — check embeddings.")
else:
    print("No valid data.")

Testing Resume ID=36856210 ↔ Job ID=Dice Id : 10110693
MATCH SUMMARY:
resume_id                                               36856210
job_id                                        Dice Id : 10110693
n_r                                                            6
n_j                                                            9
avg_match                                               0.455419
pct_covered                                             0.166667
n_matches                                                      1
gaps           [Java (computer programming), information stru...
dtype: object

TOP 3 MATCHES:


,resume_skill,job_skill,sim
0,data engineering,SQL,0.553721


In [15]:
print("Collecting unique ESCO skills from resumes and jobs...")
unique_skills = set()

# Resumes
for row in tqdm(resumes.itertuples(), total=len(resumes), desc="Resumes"):
    unique_skills.update(get_esco_skills(row))

# Jobs
for row in tqdm(jobs.itertuples(), total=len(jobs), desc="Jobs"):
    unique_skills.update(get_esco_skills(row))

unique_skills = sorted(unique_skills)
print(f"Found {len(unique_skills):,} unique ESCO skills")

print("Pre-embedding unique skills...")
new_embeddings = 0

for skill in tqdm(unique_skills, desc="Embedding"):
    if skill not in skill_cache:
        vec = embed_skill(skill)
        if vec is not None:
            skill_cache[skill] = vec
            new_embeddings += 1
        else:
            print(f"[SKIP] Failed to embed: {skill}")

print(f"Added {new_embeddings:,} new embeddings")

print(f"Saving full cache ({len(skill_cache):,} skills) → {CACHE_FILE}")
json.dump({k: v.tolist() for k, v in skill_cache.items()}, open(CACHE_FILE, "w"))
print("Pre-embedding complete!")

Resumes:   0%|          | 0/120 [00:00<?, ?it/s]

Jobs:   0%|          | 0/22000 [00:00<?, ?it/s]

Found 2,604 unique ESCO skills
Pre-embedding unique skills...


Embedding:   0%|          | 0/2604 [00:00<?, ?it/s]

Added 0 new embeddings
Saving full cache (2,604 skills) → matches\skill_cache.json
Pre-embedding complete!


In [ ]:
THRESH          = 0.45
MAX_WORKERS     = 8
BATCH_SIZE      = 1000
PAIR_TIMEOUT    = 60
SAVE_FULL_MATRIX = False
SUMMARY_FILE    = MATCH_DIR / "ind_skills_scores.parquet"

print(f"Using {len(resumes):,} tech resumes × {len(jobs):,} jobs "
      f"= {len(resumes)*len(jobs):,} total possible pairs")

print("\nBuilding valid pairs (only when both have ESCO skills)...")
pairs = []
for r in tqdm(resumes.itertuples(), total=len(resumes), desc="Scanning resumes"):
    r_skills = get_esco_skills(r)
    if not r_skills:
        continue
    for j in jobs.itertuples():
        j_skills = get_esco_skills(j)
        if j_skills:
            pairs.append((r, j))

# ────────────────────── PROGRESS SETUP ──────────────────────
total_pairs = len(pairs)
total_batches = (total_pairs + BATCH_SIZE - 1) // BATCH_SIZE
last_batch_size = total_pairs % BATCH_SIZE or BATCH_SIZE

print(f"\nVALID PAIRS FOUND     : {total_pairs:,}")
print(f"TOTAL BATCHES TO RUN  : {total_batches:,}")
print(f"BATCH SIZE            : {BATCH_SIZE:,} (last batch: {last_batch_size:,})")
print(f"{'='*70}\n")

import time
start_time = time.time()
pairs_processed = 0

def match_with_timeout(r, j):
    with ThreadPoolExecutor(max_workers=1) as exe:
        fut = exe.submit(match_resume_job, r, j, THRESH)
        try:
            return fut.result(timeout=PAIR_TIMEOUT)
        except TimeoutError:
            print(f"[TIMEOUT] Resume {getattr(r, 'ID', '?')} ↔ Job {getattr(j, 'jobid', '?')}")
            return None
        except Exception as e:
            print(f"[ERROR] Resume {getattr(r, 'ID', '?')} ↔ Job {getattr(j, 'jobid', '?')}: {e}")
            return None

print(f"\nStarting matching of {total_pairs:,} pairs in {total_batches} batches\n")
print(f"{'='*70}")

summary_rows = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as exe:
    for batch_idx in range(1, total_batches + 1):
        start = (batch_idx - 1) * BATCH_SIZE
        end = start + BATCH_SIZE
        batch = pairs[start:end]

        batch_size = len(batch)
        pairs_processed += batch_size

        print(f"BATCH {batch_idx:>3}/{total_batches} | "
              f"Pairs {start:,}→{start + batch_size - 1:,} | "
              f"Progress: {pairs_processed:,}/{total_pairs:,} "
              f"({pairs_processed/total_pairs:.2%})")

        futures = [exe.submit(match_with_timeout, r, j) for r, j in batch]

        for fut in tqdm(as_completed(futures),
                        total=len(futures),
                        desc=f"  Processing",
                        leave=False):
            result = fut.result()
            if result:
                summary_rows.append(result["summary"])

                if SAVE_FULL_MATRIX:
                    s = result["summary"]
                    pair_file = MATCH_DIR / f"pair_{s['resume_id']}_{s['job_id']}.parquet"
                    pd.DataFrame(result["sim_matrix"],
                                 index=result["r_skills"],
                                 columns=result["j_skills"]).to_parquet(pair_file)

# ────────────────────── FINAL SAVE ──────────────────────
summary_df = pd.DataFrame(summary_rows)
summary_df.to_parquet(SUMMARY_FILE, index=False)

print(f"\n{'='*70}")
print(f"COMPLETED! {len(summary_df):,} matched pairs saved to {SUMMARY_FILE}")
print(f"Total pairs processed: {total_pairs:,}")
print(f"{'='*70}")

Using 120 tech resumes × 22,000 jobs = 2,640,000 total possible pairs

Building valid pairs (only when both have ESCO skills)...


Scanning resumes:   0%|          | 0/120 [00:00<?, ?it/s]


VALID PAIRS FOUND     : 2,553,383
TOTAL BATCHES TO RUN  : 2,554
BATCH SIZE            : 1,000 (last batch: 383)


Starting matching of 2,553,383 pairs in 2554 batches

BATCH   1/2554 | Pairs 0→999 | Progress: 1,000/2,553,383 (0.04%)


  Processing:   0%|          | 0/1000 [00:00<?, ?it/s]

BATCH   2/2554 | Pairs 1,000→1,999 | Progress: 2,000/2,553,383 (0.08%)


...

BATCH 2553/2554 | Pairs 2,552,000→2,552,999 | Progress: 2,553,000/2,553,383 (99.99%)


  Processing:   0%|          | 0/1000 [00:00<?, ?it/s]

BATCH 2554/2554 | Pairs 2,553,000→2,553,382 | Progress: 2,553,383/2,553,383 (100.00%)


  Processing:   0%|          | 0/383 [00:00<?, ?it/s]


COMPLETED! 2,553,383 matched pairs saved to matches\summary_scores.parquet
Total pairs processed: 2,553,383
